# Making input data set 

In [ ]:
import pandas as pd 
import numpy as np 
import datetime
def make_series():
    time_start = datetime.datetime.strptime('20:00:00', '%H:%M:%S')
    time_end = datetime.datetime.strptime('20:00:50', '%H:%M:%S')
    time_list = []
    i = time_start 
    
    while(i<time_end):
        time_list.append(i.time())
        i = i + datetime.timedelta(seconds = 1)
    samples  = pd.Series(np.random.normal(loc=0, scale=1, size=50))
    time_list = pd.Series(time_list)
    
    columns = ['Time','Value']
    final_df = pd.DataFrame({columns[0]:time_list, columns[1]:samples})
    final_df.set_index(time_list)
    return final_df 
    

In [ ]:

X = []
for i in range(50):
    inserter = make_series()
    X.append(inserter)

print((X[0]['Value'][0]))


# Function to create sequences to be fed to LSTM

In [ ]:
def create_sequences(data,seq_length):
    x,y = [], []
    df_as_np = data.to_numpy()
    for i in range(len(data)-seq_length):
        row =  [[a] for a in df_as_np[i:i+seq_length]]
        x.append(row)
        label = df_as_np[i+5]
        y.append(label)
    return np.array(x),np.array(y)



# Loss calculater 

In [ ]:
def loss_value(true_values, predicted_values):
    error = 0
    for i in range(len(true_values)): 
        error = (true_values[i]-predicted_values[i])**2 + error 
    mean = error / len(true_values)
    error = np.sqrt(mean)
    return error
        
        


In [ ]:
import tensorflow as tf 
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense 



# Model and training 

In [ ]:
import tensorflow as tf 
from tensorflow.keras.models import Sequential 
from tensorflow.keras.layers import LSTM, Dense 
from sklearn.model_selection import train_test_split

model = Sequential()

model = Sequential()
model.add(LSTM(128,return_sequences=True, input_shape = (5,1)))
model.add(LSTM(128))
model.add(Dense(1))

model.compile(optimizer = 'adam',loss = 'mse' )

for dataset in X: 
    x,y = create_sequences(dataset['Value'],5)
    x_train, x_test_1, y_train, y_test_1 = train_test_split(x,y,test_size = 0.2,random_state = 42)
    model.fit(x,y,epochs = 10,validation_data = (x_test_1,y_test_1),batch_size=32)





# Making of test datasets 

In [ ]:
import random 
F = []
for i in range(50):
    inserter = make_series()
    F.append(inserter)


In [ ]:
G = []
for i in range(50):
    inserter = make_series()
    G.append(inserter)
    

In [ ]:
H = []
for i in range(50):
    inserter = make_series()
    H.append(inserter)

In [ ]:
#Adding anomolus spikes at random places in datasets 

for i in G: 
   
    min = 2
    max = 1
    for j in range(random.randint(1,49)):
        index_number = random.randint(1,49)
        addend = min + (max-min)*random.random()
        
        i['Value'][index_number]+= addend

In [ ]:
for i in H: 
    for j in range(50):
        i['Value'][j] += np.sin(j)
   
      

In [ ]:
def penalty(sigma, contraction_hyperparameter):
    prefac = 1/(np.pi)
    sum_terms = tf.math.atan(2*np.pi*contraction_hyperparameter*tf.abs(sigma))
    mean = np.mean(sum_terms)
    return prefac*mean

In [ ]:
# TESTING AND RECORDING OBSERVATIONS FOR F

x,y = create_sequences(F[0]['Value'],5)
y_predict = model.predict(x)
y_predict  = y_predict.reshape(y.shape)
c_0 = loss_value(y,y_predict)

c_1 = c_0

# container for storing anomolus scores 
scores_array = []

sigma = np.array([0.9,0.25,0.1,-0.5,0.80,0.23])
hyper_parameters = np.array([0.1,0.25,0.31,0.45,0.22,0.4])


score_array= abs(2*c_0 - 2*penalty(sigma,hyper_parameters)-c_1)
scores_array.append(score_array)





In [ ]:

for i in F[1:]:
    x,y = create_sequences(i['Value'],5)
    y_predict = model.predict(x)
    y_predict  = y_predict.reshape(y.shape)
    c_0 = loss_value(y,y_predict)
    score_array= abs(2*c_0 - 2*penalty(sigma,hyper_parameters)-c_1)
    c_1 = c_0
    scores_array.append(score_array)
print(len(scores_array))
    

In [ ]:
# TESTING AND RECORDING OBSERVATIONS FOR G

# container for storing anomolous scores 
scores_array_g = []
x,y = create_sequences(G[0]['Value'],5)
y_predict = model.predict(x)
y_predict  = y_predict.reshape(y.shape)
c_0 = loss_value(y,y_predict)

c_1 = c_0

sigma = np.array([0.9,0.25,0.1,-0.5,0.80,0.23])
hyper_parameters = np.array([0.1,0.25,0.31,0.45,0.22,0.4])


score_array= abs(2*c_0 - 2*penalty(sigma,hyper_parameters)-c_1)
scores_array_g.append(score_array)

In [ ]:
for i in G[1:]:
    x,y = create_sequences(i['Value'],5)
    y_predict = model.predict(x)
    y_predict  = y_predict.reshape(y.shape)
    c_0 = loss_value(y,y_predict)
    score_array= abs(2*c_0 - 2*penalty(sigma,hyper_parameters)-c_1)
    c_1 = c_0
    scores_array_g.append(score_array)

In [ ]:
# TESTING AND RECORDING OBSERVATIONS FOR H

#container for anomolous scores 
scores_array_h = []
x,y = create_sequences(H[0]['Value'],5)
y_predict = model.predict(x)
y_predict  = y_predict.reshape(y.shape)
c_0 = loss_value(y,y_predict)

c_1 = c_0

sigma = np.array([0.9,0.25,0.1,-0.5,0.80,0.23])
hyper_parameters = np.array([0.1,0.25,0.31,0.45,0.22,0.4])


score_array= abs(2*c_0 - 2*penalty(sigma,hyper_parameters)-c_1)
scores_array_h.append(score_array)

for i in H[1:]:
    x,y = create_sequences(i['Value'],5)
    y_predict = model.predict(x)
    y_predict  = y_predict.reshape(y.shape)
    c_0 = loss_value(y,y_predict)
    score_array= abs(2*c_0 - 2*penalty(sigma,hyper_parameters)-c_1)
    c_1 = c_0
    scores_array_h.append(score_array)